In [ ]:
%pip install --quiet langchain langgraph tavily-python langchain-ollama

In [2]:
user_input = "Write a report on the impact of climate change on polar bears."

In [3]:
import os
import dotenv

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
import operator
from pydantic import BaseModel
from typing import List
from typing_extensions import Annotated

class QueryResult(BaseModel):
    title: str = None
    url: str = None
    resume: str = None
    

class ReportState(BaseModel):
    user_input: str = None
    final_response: str = None
    queries: List[str] = []
    queries_results: Annotated[List[QueryResult], operator.add]

In [5]:
reasoning = {
    "effort": "medium",  # 'low', 'medium', or 'high'
    "summary": "auto",  # 'detailed', 'auto', or None
}

In [6]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

reasoning_llm_deepseek = ChatOllama(
    model = "deepseek-r1:1.5b",
)

reasoning_llm_openai = ChatOpenAI(
    model="o4-mini", reasoning=reasoning
)

llm = ChatOpenAI(model="gpt-4.1")

In [7]:
agent_prompt = """
You are a research planner agent
You are  working on a project that aims to answer user's question
using resources found online

Your asnwer should be technical, detailed and well structured using up to date information
Cite facts, data and specific informations.

Here is the user input
<USER_INPUT>
{user_input}
</USER_INPUT>
"""

In [8]:
build_queries = agent_prompt + """
Your first objective is to build a list of queries that
will be used to find answers to the user's question.

Answer with anything between 2 and 3 queries.
"""

In [9]:
resume_search = agent_prompt + """
Your objective here is to analyze the web search results and make a synthesis of it.
Emphasize the most relevant information based on user's question.

After your work, another agent will use the synthesis to build a final response to the user,
so make sure the synthesis contains only useful information.

Be concise and clear, do not preamble.

Here is the web search results:
<SEARCH_RESULTS>
{search_results}
</SEARCH_RESULTS>
"""

In [10]:
build_final_response = agent_prompt + """
Your objective here is to develop a final response to the user using the reports made during
the web search, with their syntesis.

The response should contain something between 3 and 5 paragraphs.

Here is the web search results:
<SEARCH_RESULTS>
{search_results}
</SEARCH_RESULTS>

You must add reference citations with the number of the citation (e.g. [1], [2], etc.) at the end of each paragraph.
and the articles you used in each paragraph of your answer.
"""

In [11]:
def build_first_queries(state: ReportState):
    class QueryList(BaseModel):
        queries: List[str]
    
    user_input = state.user_input
    prompt = build_queries.format(user_input=user_input)
    query_llm = llm.with_structured_output(QueryList)

    result = query_llm.invoke(prompt)
    
    print("Generated queries from Query Builder: \n {result}\n", result)

    return{"queries": result.queries} #Formatted queries
    

In [12]:
from langgraph.types import Send

def spawn_researchers(state: ReportState):
    return [Send("single_search", query) for query in state.queries]

In [13]:
from tavily import TavilyClient
#debug this
def single_search(query: str):
    tavily_client = TavilyClient()
    results = tavily_client.search(query, 
                                   max_results=1,
                                   include_raw_content=False)
    
    url = results["results"][0]["url"]
    url_extraction = tavily_client.extract(url)

    if len(url_extraction["results"]) > 0:
        raw_content = url_extraction["results"][0]["raw_content"]
        prompt = resume_search.format(user_input=user_input, search_results=raw_content)
        llm_result = llm.invoke(prompt)

        query_results = QueryResult(title=results["results"][0]["title"],
                                   url=url,
                                   resume=llm_result.content)
        
        print("\n\nAgent for query:", query)
        print("Result:", query_results)
        
        return{"queries_results": [query_results]}

In [14]:
def final_writer(state: ReportState):
    search_results = ""
    reference = ""
    for i, result in enumerate(state.queries_results):
        search_results += f"[{i+1}]\n\n"
        search_results += f"Title: {result.title}\n"
        search_results += f"URL: {result.url}\n"
        search_results += f"Content: {result.resume}\n\n"

        reference += f"[{i+1}] - {result.title} ({result.url})\n"
    
    prompt = build_final_response.format(user_input=state.user_input, search_results=search_results)
    print("\n\nFinal Writer Prompt:\n", prompt)

    llm_result = llm.invoke(prompt)

    final_response = llm_result + "\n\nReferences:\n" + reference

    return {"final_response": final_response}

In [15]:
from langgraph.graph import START, END, StateGraph


builder = StateGraph(ReportState)
builder.add_node("build_first_queries", build_first_queries)
builder.add_node("single_search", single_search)
builder.add_node("final_writer", final_writer)

builder.add_edge(START, "build_first_queries")
builder.add_conditional_edges("build_first_queries", spawn_researchers, ["single_search"])
builder.add_edge("single_search", "final_writer")
builder.add_edge("final_writer", END)

graph = builder.compile()

In [ ]:
response = graph.invoke({"user_input": user_input})

In [ ]:
print(response['final_response'].messages[0].content)